# import

In [ ]:
import requests
import pandas as pd
import json
import os
from datetime import datetime, timedelta

import openmeteo_requests

import requests_cache
from retry_requests import retry

import plotly.express as px
import plotly.offline as pyo



Test de l'adresse avec un exemple de ville

In [3]:
r = requests.get('https://nominatim.openstreetmap.org/search?addressdetails=1&city=Paris&json=json')
r

<Response [200]>

Initialisation de la liste des villes

In [4]:
city = ["Le Mont Saint Michel",
"Saint Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau-du-Haut-Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"La palud sur Verdon",
"Bormes-les-Mimosas",
"Cassis",
"Marseille",
"Aix-en-Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues-Mortes",
"Saintes-Maries-de-la-mer",
"Collioure",
"Carcassonne",
"Foix",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]


In [5]:
# Pour obtenir  une vision plus facile, je mets des définitions aux code météos OMM en limitant les possibilités
weather_code_map = {
    0: "Soleil",
    1: "Soleil",
    2: "Soleil",
    15: "Soleil",
    16: "Nuageux",
    45: "Nuageux",
    48: "Nuageux",
    51: "Bruine",
    53: "Bruine",
    55: "Bruine",
    56: "Bruine",
    57: "Bruine",
    61: "Pluie",
    63: "Pluie",
    65: "Pluie",
    66: "Pluie",
    67: "Pluie",
    71: "Neige",
    73: "Neige",
    75: "Neige",
    77: "Neige",
    80: "Averses",
    81: "Averses",
    82: "Averses",
    85: "Averses",
    86: "Averses",
    95: "Orage",
    96: "Orage",
    99: "Orage"
}

weather_code_map_cplt = {}
last_value = None
for i in range(100):  # de 0 à 99 inclus
    if i in weather_code_map:
        last_value = weather_code_map[i]
    elif last_value is None:
        # avant la première valeur (ex: i < 0), on met la première valeur connue
        last_value = list(weather_code_map.values())[0]
    weather_code_map_cplt[i] = last_value

weather_map_inv = {
    "Soleil": 1,
    "Nuageux": 2,
    "Bruine": 3,
    "Pluie": 4,
    "Neige": 5,
    "Averses": 6,
    "Orage": 7
}
weather_map = {
    1: "Soleil",
    2: "Nuageux",
    3: "Bruine",
    4: "Pluie",
    5: "Neige",
    6: "Averses",
    7: "Orage"
}

In [ ]:
# Adresse de l'API pour obtenir les coordonnées des villes
url_city = "https://nominatim.openstreetmap.org/search"

today = datetime.now()
start_date = (today + timedelta(days=4)).strftime('%Y-%m-%d')
end_date = (today + timedelta(days=11)).strftime('%Y-%m-%d')

# Configuration du client Open-Meteo API avec cache et réessai en cas d'erreur
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Adresse de l'API pour la météo
url_meteo = "https://api.open-meteo.com/v1/forecast"

villes = []
meteos = []
id = 0
for ville in city:
    id += 1
    params_nominatim = {
        "format": "json",
        "limit": "1",
        "email": "ristou@free.fr",
        "country": "France",
        "city": ville
    }

    try:
        response_city = requests.get(url_city, params=params_nominatim)
        response_city.raise_for_status()  # Lève une exception pour les erreurs HTTP
        r = response_city.json()

        if r:
            print("ok city", ville)
            villes.append({
                "ville_id": id,
                "nom": ville,
                "latitude": r[0]["lat"],
                "longitude": r[0]["lon"]
            })

            params_meteo = {
                "latitude": r[0]["lat"],
                "longitude": r[0]["lon"],
                "daily": ["weather_code", "apparent_temperature_max", "sunshine_duration", "precipitation_sum", "precipitation_probability_max", "wind_speed_10m_max"],
                "timezone": "auto",
                "start_date": start_date,
                "end_date": end_date,
            }

            responses = openmeteo.weather_api(url_meteo, params=params_meteo)
            if responses:
                response = responses[0]
                daily = response.Daily()

                # Générer les dates
                dates = pd.date_range(
                    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=daily.Interval()),
                    inclusive="left")

                # Récupérer les valeurs météorologiques
                weather_code = daily.Variables(0).ValuesAsNumpy()
                temp_max = daily.Variables(1).ValuesAsNumpy()
                sunshine_duration = daily.Variables(2).ValuesAsNumpy()
                precipitation_sum = daily.Variables(3).ValuesAsNumpy()
                precipitation_prob = daily.Variables(4).ValuesAsNumpy()
                wind_speed = daily.Variables(5).ValuesAsNumpy()

                # Créer une ligne pour chaque date
                for i, date in enumerate(dates):
                    meteos.append({
                        "ville_id": id,
                        "date": date.date(),
                        "condition": weather_code[i],
                        "temperature": temp_max[i],
                        "duree_ensoleillement": sunshine_duration[i],
                        "pluie": precipitation_sum[i],
                        "vent": wind_speed[i]
                    })
            else:
                print("erreur météo : ", ville)
        else:
            print("erreur city : ", ville)
    except requests.exceptions.RequestException as e:
        print(f"Erreur de requête pour {ville}: {e}")

df_villes = pd.DataFrame(villes)
df_meteos = pd.DataFrame(meteos)

In [ ]:
for v in villes:
    params_reverse = {
        "lat": v["latitude"],
        "lon": v["longitude"],
        "format": "json",
        "zoom": 10,
        "addressdetails": 1,
        "email": "ristou@free.fr"
    }

    try:
        response_rev = requests.get("https://nominatim.openstreetmap.org/reverse", params=params_reverse)
        response_rev.raise_for_status()

        # Vérifier si la réponse contient bien du JSON
        if "application/json" not in response_rev.headers.get("Content-Type", ""):
            print(f"⚠️ Réponse non JSON pour {v['nom']} : {response_rev.text[:100]}")
            continue

        data_rev = response_rev.json()

        city_name = data_rev.get("address", {}).get("city") or \
                    data_rev.get("address", {}).get("town") or \
                    data_rev.get("address", {}).get("village")

        if city_name:
            if city_name.lower() != v["nom"].lower():
                print(f"Erreur possible : {v['nom']} vs {city_name}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur réseau pour {v['nom']}: {e}")
    except ValueError:
        print(f"❌ Erreur JSON vide pour {v['nom']}: {response_rev.text[:100]}")


🔍 Vérif : Le Mont Saint Michel → Le Mont-Saint-Michel
⚠️ Mismatch possible : Le Mont Saint Michel vs Le Mont-Saint-Michel
🔍 Vérif : Saint Malo → Saint-Malo
⚠️ Mismatch possible : Saint Malo vs Saint-Malo
🔍 Vérif : Bayeux → Bayeux
🔍 Vérif : Le Havre → Le Havre
🔍 Vérif : Rouen → Rouen
🔍 Vérif : Paris → Paris
🔍 Vérif : Amiens → Amiens
🔍 Vérif : Lille → Lille
🔍 Vérif : Strasbourg → Strasbourg
🔍 Vérif : Chateau-du-Haut-Koenigsbourg → Orschwiller
⚠️ Mismatch possible : Chateau-du-Haut-Koenigsbourg vs Orschwiller
🔍 Vérif : Colmar → Colmar
🔍 Vérif : Eguisheim → Eguisheim
🔍 Vérif : Besancon → Besançon
⚠️ Mismatch possible : Besancon vs Besançon
🔍 Vérif : Dijon → Dijon
🔍 Vérif : Annecy → Annecy
🔍 Vérif : Grenoble → Grenoble
🔍 Vérif : Lyon → Lyon
🔍 Vérif : La palud sur Verdon → La Palud-sur-Verdon
⚠️ Mismatch possible : La palud sur Verdon vs La Palud-sur-Verdon
🔍 Vérif : Bormes-les-Mimosas → Bormes-les-Mimosas
🔍 Vérif : Cassis → Cassis
🔍 Vérif : Marseille → Marseille
🔍 Vérif : Aix-en-Provence → 

Après vérification Orschwiller est à côté du Château du Haut Koenigsbourg

In [ ]:
df_villes['latitude2'] = pd.to_numeric(df_villes['latitude'], errors='coerce')
df_villes['longitude2'] = pd.to_numeric(df_villes['longitude'], errors='coerce')

fig = px.scatter_map(
    df_villes,
    lat="latitude2",
    lon="longitude2",
    hover_name="nom",
    hover_data={
        "ville_id": False,
        "nom": True,
        "latitude2": False,
        "longitude2": False
    },
    zoom=4,
    title="Emplacement des villes"
)

fig.show()


In [ ]:
df_meteos["condition"]=df_meteos["condition"].map(weather_code_map_cplt).map(weather_map_inv)
df_meteos

In [11]:
df_villes.to_csv("data//villes.csv", index=False,sep=';')

df_meteos.to_csv("data//meteos.csv", index=False, sep=';')

df_condition = pd.DataFrame(list(weather_map_inv.items()), columns=["description", "code"])
df_condition.to_csv("data//condition.csv", index=False, sep=';')


Lancement du scrapping du site booking

Sélection de la même période. Je limite les chambres entre 100 et 250€ la nuit.

In [ ]:
#Lancement 1 par 1 pour réduire les délais
for i in range(len(df_villes)):
    df_villes[['nom','latitude','longitude']][i:i+1].to_json("villes.json", orient="records", force_ascii=False, indent=4)
    !python booking_lat_lon.py villes.json


Conversion des fichiers json en dataframe puis csv

In [ ]:
# Chemin du répertoire contenant les fichiers JSON
repertoire = 'booking_results'

# Liste pour stocker les DataFrames
dataframes = []

# Parcourir les fichiers dans le répertoire
for fichier in os.listdir(repertoire):
    if fichier.endswith('.json'):
        chemin_fichier = os.path.join(repertoire, fichier)
        try:
            with open(chemin_fichier, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Convertir les données JSON en DataFrame
                df = pd.DataFrame(data)
                dataframes.append(df)
        except Exception as e:
            print(f"Erreur avec le fichier {fichier}: {e}")


# Concaténer tous les DataFrames en un seul
df_combine = pd.concat(dataframes, ignore_index=True)

# Fusionner les deux DataFrames
df_hotels = pd.merge(
    df_combine,
    df_villes[['nom', 'ville_id']],
    left_on='ville', 
    right_on='nom',
    how='left'
)

df_hotels.drop('nom', axis=1, inplace=True)

# Afficher le DataFrame combiné avec ville_id
df_hotels


In [15]:
df_hotels.to_csv("data//hotels.csv", index=False, sep=';', encoding='utf-8-sig')


In [16]:
# Calcul de la moyenne par ville
df_meteos_moyenne = df_meteos.groupby('ville_id').mean(numeric_only=True).round(0).reset_index()

# Calcul de la description la plus fréquente par ville
df_meteos_mode = df_meteos.groupby('ville_id')['condition'].apply(lambda x: x.mode()[0]).reset_index()

# Fusionner les deux DataFrames
df_meteos_general = pd.merge(df_meteos_moyenne, df_meteos_mode, on='ville_id')

df_meteos_general['condition_label']=df_meteos_general['condition_x'].map(weather_map)

In [17]:
# Fusionner les DataFrames
df_merged = pd.merge(df_villes, df_meteos_general, on='ville_id')

df_merged['latitude'] = pd.to_numeric(df_merged['latitude'], errors='coerce')
df_merged['longitude'] = pd.to_numeric(df_merged['longitude'], errors='coerce')

df_merged

# Créer la carte pour les températures moyennes
map_temp_moy = px.scatter_map(
    df_merged,
    lat="latitude",
    lon="longitude",
    size="temperature",
    hover_name="nom",
    hover_data={
        "latitude": False,
        "longitude": False,
        "temperature": True,
        "condition_label": True,
        "pluie":True,
        "vent":True
    },
    color="temperature",
    color_continuous_scale="Bluered",
    zoom=4,
    map_style="open-street-map",
    title="Température moyenne séjour"
)

# Sauvegarder et afficher la carte
pyo.plot(map_temp_moy, filename='image//carte_temp_moy.html', auto_open=True)


'image//carte_temp_moy.html'